In [7]:
print("Hello Speckle Tutorial Get Block Autocad!")

Hello Speckle Tutorial Get Block Autocad!


In [8]:
# Install update Specklepy 
#!pip install --upgrade pip
#!pip install --upgrade specklepy

In [9]:
# Import Library Specklepy
import specklepy
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account

In [10]:
# Get your account info
account = get_default_account()
print("Name of account: " + account.userInfo.name)
client = SpeckleClient(host="speckle.xyz")
client.authenticate_with_account(account)

Name of account: Hồ Văn Chương


In [11]:
# Get your stream sent from Autocad
# url = https://speckle.xyz/streams/088f1b9f0d/commits/a5a06e4900
streamId = "088f1b9f0d" 
stream = client.stream.get(streamId)
print("Name of stream: " + stream.name)

Name of stream: AutocadTest


In [12]:
# Get all commit from your stream 
# get list of commits
commits = client.commit.list(streamId)
print("Number of commits: " + str(len(commits)))
# Get specific commit
commitId = "a5a06e4900"
commit = client.commit.get(streamId, commitId)


Number of commits: 7


In [13]:
from specklepy.transports.server import ServerTransport
from specklepy.api import operations
transport = ServerTransport(client=client, stream_id=streamId)
res = operations.receive(commit.referencedObject, transport)
blocks = res["@Blocks"]

In [14]:
# Get all blocks from your stream
print("Number of blocks: " + str(len(blocks)))

Number of blocks: 17


In [15]:
# Create a dictionary with information of blocks
data = []
for block in blocks:
    dictBlocks = {}
    dictBlocks["Name"] = block.definition.name
    dictBlocks["X"] = block.transform.matrix[3]
    dictBlocks["Y"] = block.transform.matrix[7]
    dictBlocks["Z"] = block.transform.matrix[11]
    data.append(dictBlocks)
# Create a dataframe with information of blocks
import pandas as pd
df = pd.DataFrame(data)
df


,Name,X,Y,Z
0,Ponto de Força Saida de Fio,0.535000,3.194069,0.000000
1,Ponto de Força Saida de Fio,4.110000,6.765946,0.000000
2,a.Simples,1.346186,4.294069,0.000000
3,a.Simples,2.423730,0.164062,0.000000
4,a.Simples,3.614062,2.288337,0.000000
5,No host family,6.679914,4.881626,0.000000
6,No host family,6.679914,2.843446,0.000000
7,No host family,4.751273,2.339459,0.000000
8,No host family,4.736437,4.073766,0.000000
9,No host family,3.037749,3.577191,0.000000


In [16]:
# Save dataframe to csv file
df.to_csv("blocks.csv", index=False)


In [17]:
mat = [ [0.9396926 ,-0.342020154,0,957.49,],
       [0.342020154 ,0.9396926, 0, 15.0643969],
       [0,0,1,0],
       [0,0,0,1]
       ]
def getLocation(mat):
    x = mat[0][3]
    y = mat[1][3]
    z = mat[2][3]
    return x,y,z
print(getLocation(mat))
def getAngle(mat):
    x = mat[0][0]
    y = mat[1][0]
    z = mat[2][0]
    return x,y,z
print(getAngle(mat))

(957.49, 15.0643969, 0)
(0.9396926, 0.342020154, 0)


In [18]:
import numpy as np
import math

def create_transform_matrix(x, y, z, rotation_angles):
    # Convert rotation angles to radians
    rotation_angles_radians = np.radians(rotation_angles)
    roll, pitch, yaw = rotation_angles_radians

    # Create rotation matrices for each axis
    rotation_x = np.array([
        [1, 0, 0, 0],
        [0, math.cos(roll), -math.sin(roll), 0],
        [0, math.sin(roll), math.cos(roll), 0],
        [0, 0, 0, 1]
    ])

    rotation_y = np.array([
        [math.cos(pitch), 0, math.sin(pitch), 0],
        [0, 1, 0, 0],
        [-math.sin(pitch), 0, math.cos(pitch), 0],
        [0, 0, 0, 1]
    ])

    rotation_z = np.array([
        [math.cos(yaw), -math.sin(yaw), 0, 0],
        [math.sin(yaw), math.cos(yaw), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    # Create the translation matrix
    translation = np.array([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

    # Combine rotation and translation matrices
    transform_matrix = np.dot(translation, np.dot(rotation_z, np.dot(rotation_y, rotation_x)))

    return transform_matrix

# Example input
x = 957.49
y = 15.0643969 
z = 0
rotation_angles = (20, 0, 0)  # Roll = 20 degrees

# Create and print the transformation matrix
transform_matrix = create_transform_matrix(x, y, z, rotation_angles)
print("Transformation Matrix:\n", transform_matrix)

Transformation Matrix:
 [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  9.57490000e+02]
 [ 0.00000000e+00  9.39692621e-01 -3.42020143e-01  1.50643969e+01]
 [ 0.00000000e+00  3.42020143e-01  9.39692621e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [19]:
import math
import numpy as np

def create_transformation_matrix(x, y, z, angle):
    # Convert the angle to radians
    angle_rad = math.radians(angle)
    
    # Create the rotation matrix
    rotation_matrix = np.array([[math.cos(angle_rad), -math.sin(angle_rad), 0, 0],
                                [math.sin(angle_rad), math.cos(angle_rad), 0, 0],
                                [0, 0, 1, 0],
                                [0, 0, 0, 1]])
    
    # Create the translation matrix
    translation_matrix = np.array([[1, 0, 0, x],
                                   [0, 1, 0, y],
                                   [0, 0, 1, z],
                                   [0, 0, 0, 1]])
    
    # Compute the transformation matrix
    transformation_matrix = np.dot(translation_matrix, rotation_matrix)
    
    return transformation_matrix
x = 957.49
y = 15.0643969
z = 0
angle = 20

transformation_matrix = create_transformation_matrix(x, y, z, angle)
print(transformation_matrix)


[[ 9.39692621e-01 -3.42020143e-01  0.00000000e+00  9.57490000e+02]
 [ 3.42020143e-01  9.39692621e-01  0.00000000e+00  1.50643969e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [20]:
import numpy as np
import math

def get_rotation_angles(matrix):
    # Extract the rotation submatrix
    rotation_matrix = matrix[:3, :3]

    # Calculate the rotation angles using the axis-angle representation
    axis_angle = rotation_matrix_to_axis_angle(rotation_matrix)
    rotation_angle = np.linalg.norm(axis_angle)
    rotation_axis = axis_angle / rotation_angle

    # Convert the rotation angle to degrees
    rotation_angle_degrees = math.degrees(rotation_angle)

    return rotation_axis, rotation_angle_degrees

def rotation_matrix_to_axis_angle(matrix):
    trace = np.trace(matrix)
    angle = math.acos((trace - 1) / 2.0)

    axis = np.array([
        matrix[2, 1] - matrix[1, 2],
        matrix[0, 2] - matrix[2, 0],
        matrix[1, 0] - matrix[0, 1]
    ])

    axis /= (2 * math.sin(angle))

    return axis * angle

# Example matrix
matrix = np.array([
    [0.9396926, -0.342020154, 0, 957.49],
    [0.342020154, 0.9396926, 0, 15.0643969],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])

# Calculate and print the rotation angles
print("Test with Example:")
rotation_axis, rotation_angle = get_rotation_angles(matrix)
print("Rotation Axis:", rotation_axis)
print("Rotation Angle (Degrees):", round(rotation_angle, 2))

# Test with result transformation
print("Test result transformation:")
# Calculate and print the rotation angles
rotation_axis, rotation_angle = get_rotation_angles(transformation_matrix)
print("Rotation Axis:", rotation_axis)
print("Rotation Angle (Degrees):", round(rotation_angle, 2))

Test with Example:
Rotation Axis: [0. 0. 1.]
Rotation Angle (Degrees): 20.0
Test result transformation:
Rotation Axis: [0. 0. 1.]
Rotation Angle (Degrees): 20.0
